In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
# Importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

## Data collection and processing

In [3]:
# Loading data into df
heart_data = pd.read_csv('datasets/heart.csv')

In [4]:
# First 5 rows
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
# Number of rows and columns
heart_data.shape

(303, 14)

In [7]:
# Checking for missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
# Checking distribution of target variable
heart_data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

1 $\longrightarrow$ Defective heart   
0 $\longrightarrow$ Healthy heart 

## Splitting the features and the target

In [9]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [10]:
print(X)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [11]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


## Train test split

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=3)

In [39]:
print(X.shape, X_train.shape, X_test.shape)

(303, 13) (242, 13) (61, 13)


## Comparing the performance of the models - Train test split

In [40]:
# List of models
# Default max_iter for Logistic Regression = 100, with this we will get a warning because we didn't standardize the data
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier()]

In [41]:
def compare_models_train_test():
    for model in models:
        # training the model
        model.fit(X_train, Y_train)
        # evaluating the model
        test_data_prediction = model.predict(X_test)
        accuracy = accuracy_score(Y_test, test_data_prediction)
        print(f'Accuracy score of the model {model}: {accuracy}')

In [42]:
compare_models_train_test()

Accuracy score of the model LogisticRegression(max_iter=1000): 0.7868852459016393
Accuracy score of the model SVC(kernel='linear'): 0.7704918032786885
Accuracy score of the model KNeighborsClassifier(): 0.6557377049180327
Accuracy score of the model RandomForestClassifier(): 0.7868852459016393


## Cross Validation

**Logistic Regression**

In [48]:
# CV: int -> uses StratifiedKFold, which is recommended: 
# Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
cv_score_lr = cross_val_score(LogisticRegression(max_iter=1000), X, Y, cv=5)
# Print cv score
print(cv_score_lr)
# Mean Accuracy
mean_accuracy_lr = np.mean(cv_score_lr)
# Mean accurace percentage
mean_accuracy_lr_percentage = round(mean_accuracy_lr * 100, 2)
print(mean_accuracy_lr_percentage)

[0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
82.83


**Support Vector Classifier**

In [49]:
# CV: int -> uses StratifiedKFold, which is recommended: 
# Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
cv_score_svc = cross_val_score(SVC(kernel='linear'), X, Y, cv=5)
# Print cv score
print(cv_score_svc)
# Mean Accuracy
mean_accuracy_svc = np.mean(cv_score_svc)
# Mean accurace percentage
mean_accuracy_svc_percentage = round(mean_accuracy_svc * 100, 2)
print(mean_accuracy_svc_percentage)

[0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
82.83


The same can be done with the other models, but instead we are going to create a function for all of them at the same time

## Comparing the performance of the models - Cross Validation

In [57]:
# Default max_iter for Logistic Regression = 100, with this we will get a warning because we didn't standardize the data
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier()]

In [72]:
def compare_models_cross_validation():
    for model in models:
        cv_score = cross_val_score(model, X, Y, cv=5)
        # Mean Accuracy
        mean_accuracy = np.mean(cv_score)
        # Mean accurace percentage
        mean_accuracy_percentage = round(mean_accuracy * 100, 2)
        print(f'Cross Validation Accuracies for {model} = {cv_score}')
        print(f'Mean accuracy for model {model} = {mean_accuracy_percentage}')
        print('-' * 120)

In [73]:
compare_models_cross_validation()

Cross Validation Accuracies for LogisticRegression(max_iter=1000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Mean accuracy for model LogisticRegression(max_iter=1000) = 82.83
------------------------------------------------------------------------------------------------------------------------
Cross Validation Accuracies for SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Mean accuracy for model SVC(kernel='linear') = 82.83
------------------------------------------------------------------------------------------------------------------------
Cross Validation Accuracies for KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Mean accuracy for model KNeighborsClassifier() = 64.39
------------------------------------------------------------------------------------------------------------------------
Cross Validation Accuracies for RandomForestClassifier() = [0.81967213 0.8852459  0.78688525 0.85       0.766666